In [2]:
# Dependencies
import pandas as pd

from bs4 import BeautifulSoup
import requests
import pymongo

from sqlalchemy import create_engine
from config import username
from config import password

# Datasource 1

In [3]:
# CSV for 1)	Data source 1 – Alphabetical List of Western Australian Schools (XLSX)

# Store CSV into df
csv_file = "Resources/WASchoolsList_cleaned.csv"
WASchools_df = pd.read_csv(csv_file)
WASchools_df.head()

,Code,School Name,Street,Suburb,State,Postcode,Postal Street,Postal Suburb,Postal State,Postal Postcode,...,Y06,UPR,Y07,Y08,Y09,Y10,Y11,Y12,USE,Total Students
0,5001,ADAM ROAD PRIMARY SCHOOL,HOTCHIN STREET,SOUTH BUNBURY,WA,6230,HOTCHIN STREET,SOUTH BUNBURY,WA,6230,...,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,446
1,1463,AL-AMEEN COLLEGE,57 SOUTHGATE ROAD,LANGFORD,WA,6147,57 SOUTHGATE ROAD,LANGFORD,WA,6147,...,96.0,NaN,83.0,79.0,66.0,67.0,48.0,45.0,NaN,"1,136"
2,1397,AL-HIDAYAH ISLAMIC SCHOOL,CNR HEDLEY ST/NYAMUP WAY,BENTLEY,WA,6102,CNR HEDLEY ST/NYAMUP WAY,BENTLEY,WA,6102,...,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,202
3,8001,ALBANY COMMUNITY KINDERGARTEN,136 SERPENTINE ROAD,ALBANY,WA,6330,PO BOX 5670,ALBANY,WA,6332,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29
4,5002,ALBANY PRIMARY SCHOOL,SUFFOLK STREET,ALBANY,WA,6330,SUFFOLK STREET,ALBANY,WA,6330,...,58.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,426


In [4]:
# look at all the columns available
WASchools_df.columns

Index(['Code', 'School Name', 'Street', 'Suburb', 'State', 'Postcode',
       'Postal Street', 'Postal Suburb', 'Postal State', 'Postal Postcode',
       'Latitude', 'Longitude', 'Phone', 'Education Region',
       'Broad Classification', 'Classification Group', 'Low Year', 'High Year',
       'KIN', 'PPR', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'UPR', 'Y07',
       'Y08', 'Y09', 'Y10', 'Y11', 'Y12', 'USE', 'Total Students'],
      dtype='object')

In [5]:
# create new data with only relevant columns
new_WASchools_df = WASchools_df[['School Name', 'Classification Group', 'Y12']].copy()
new_WASchools_df.head()


,School Name,Classification Group,Y12
0,ADAM ROAD PRIMARY SCHOOL,PRIMARY SCHOOLS,NaN
1,AL-AMEEN COLLEGE,NON-GOVERNMENT,45.0
2,AL-HIDAYAH ISLAMIC SCHOOL,NON-GOVERNMENT,NaN
3,ALBANY COMMUNITY KINDERGARTEN,COMMUNITY KINDERGARTENS,NaN
4,ALBANY PRIMARY SCHOOL,PRIMARY SCHOOLS,NaN


In [6]:
# remove instances where Y12 is NaN - i.e. dataset is only for secondary schools with Y12 students in it
nan_value = float("NaN")
new_WASchools_df.replace("", nan_value, inplace=True)
new_WASchools_df.dropna(subset = ["Y12"], inplace=True)

# create id
new_WASchools_df.insert(0, 'School_ID', range(0, 0 + len(new_WASchools_df)))

# rename columns
new_WASchools_df = pd.DataFrame(new_WASchools_df).rename(columns = {'School Name':'School_Name'})
new_WASchools_df = pd.DataFrame(new_WASchools_df).rename(columns = {'Classification Group':'Classification_Group'})

new_WASchools_df.head()


,School_ID,School_Name,Classification_Group,Y12
1,0,AL-AMEEN COLLEGE,NON-GOVERNMENT,45.0
5,1,ALBANY SECONDARY EDUCATION SUPPORT CENTRE,EDUCATION SUPPORT,17.0
6,2,ALBANY SENIOR HIGH SCHOOL,SECONDARY SCHOOLS,130.0
8,3,ALKIMOS BAPTIST COLLEGE,NON-GOVERNMENT,12.0
12,4,ALL SAINTS' COLLEGE,NON-GOVERNMENT,144.0


In [7]:
# get unique classification groups
classification = new_WASchools_df["Classification_Group"].unique()
classification_df = pd.DataFrame(classification).rename(columns = {0:'Classification_Group'})
# create id
classification_df.insert(0, 'Classification_ID', range(0, 0 + len(classification_df)))
classification_df.head()


,Classification_ID,Classification_Group
0,0,NON-GOVERNMENT
1,1,EDUCATION SUPPORT
2,2,SECONDARY SCHOOLS
3,3,K-12 SCHOOLS
4,4,DISTRICT HIGH SCHOOLS


In [8]:
#  Replace the Classification Group with the ID from the primary table
new_WASchools_df2 = pd.merge(new_WASchools_df, classification_df, on="Classification_Group", how="left").drop("Classification_Group", axis=1)
new_WASchools_df2

,School_ID,School_Name,Y12,Classification_ID
0,0,AL-AMEEN COLLEGE,45.0,0
1,1,ALBANY SECONDARY EDUCATION SUPPORT CENTRE,17.0,1
2,2,ALBANY SENIOR HIGH SCHOOL,130.0,2
3,3,ALKIMOS BAPTIST COLLEGE,12.0,0
4,4,ALL SAINTS' COLLEGE,144.0,0
...,...,...,...,...
307,307,WOODVALE SECONDARY COLLEGE,212.0,2
308,308,WYNDHAM DISTRICT HIGH SCHOOL,6.0,4
309,309,YANCHEP SECONDARY COLLEGE,70.0,2
310,310,YIRAMALAY/WESLEY STUDIO SCHOOL,11.0,0


In [9]:
new_WASchools_df2.loc[new_WASchools_df2['School_Name'] == "APPLECROSS SENIOR HIGH SCHOOL"]

,School_ID,School_Name,Y12,Classification_ID
6,6,APPLECROSS SENIOR HIGH SCHOOL,286.0,2


# Datasource 2

In [10]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [11]:
# Define database and collection
db = client.schools_db
collection = db.applecross

In [12]:
# URL of page to be scraped - Data source 1 – REIWA Applecross suburb profile, secondary schools
url = 'https://reiwa.com.au/suburb/applecross/'

# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

print(soup)

<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]--><!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8"> <![endif]--><!--[if IE 8]>         <html class="no-js lt-ie9"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en">
<!--<![endif]-->
<head><title>
	Applecross Suburb Profile | Property Market, House Prices and More - REIWA
</title><meta charset="utf-8"/><meta content="width=device-width, initial-scale=1.0" name="viewport"/><meta content="Interested in buying, renting or investing in Applecross? View median house prices, growth rates, demographics, schools and more with our suburb profile." name="description"/><meta content="REIWA" name="author"/><link href="/WorkArea/FrameworkUI/css/ektron.stylesheet.ashx?id=-991207082+-409962787+-2046276955+-1495356069+490163735+-719310687+-384744968+-987815539+2007979625+-2081147131+-545874270+1317633679+-1252017282+-707468958+-2117417196+-1057278288+1510350268+-559785714+-243909663+1684155

In [13]:
# Examine the results, then determine element that contains sought info
results = soup.find_all('td', class_='text-left col-xs-8')[4]('span')
print(results)

[<span id="ctl00_uxContentHolder_dzMainMid_uxColumnDisplay_ctl00_uxControlColumn_ctl00_uxWidgetHost_uxWidgetHost_widget_ctl00_StatsOtherNearbySchools"> Applecross Senior High School, Aquinas College, Santa Maria College, Como Secondary College, Melville Senior High School, Corpus Christi College, Penrhos College, Rossmoyne Senior High School, All Saints' College, Wesley College</span>]


In [14]:
# pick out only the data elements
data_schools = [element.text for element in results][0]
print(data_schools)

 Applecross Senior High School, Aquinas College, Santa Maria College, Como Secondary College, Melville Senior High School, Corpus Christi College, Penrhos College, Rossmoyne Senior High School, All Saints' College, Wesley College


In [15]:
split_data=data_schools.split(',')
print(split_data)

[' Applecross Senior High School', ' Aquinas College', ' Santa Maria College', ' Como Secondary College', ' Melville Senior High School', ' Corpus Christi College', ' Penrhos College', ' Rossmoyne Senior High School', " All Saints' College", ' Wesley College']


In [16]:
# Add the list of schools to a dataframe for filtering
# rename column
applecross_df = pd.DataFrame(split_data).rename(columns = {0:'School_Name'})

# make it upper case for consistency with other datasets
applecross_df['School_Name']=applecross_df['School_Name'].str.upper()

applecross_df.head()

,School_Name
0,APPLECROSS SENIOR HIGH SCHOOL
1,AQUINAS COLLEGE
2,SANTA MARIA COLLEGE
3,COMO SECONDARY COLLEGE
4,MELVILLE SENIOR HIGH SCHOOL


In [61]:
applecross_df['School_Name']=applecross_df['School_Name'].str.strip()
new_WASchools_df["School_Name"]= new_WASchools_df.School_Name.str.strip()

In [76]:
# merge in school_id
applecross_df2= pd.merge(applecross_df, new_WASchools_df, on="School_Name", how = "left").drop(["Classification_Group","Y12","School_Name"],axis=1)
applecross_df2.head()



,School_ID
0,6
1,7
2,244
3,63
4,188


# Datasource 3

In [18]:
# <!-- URL for Data source 3 – WA School Ranking – 2020 -->
url = 'https://bettereducation.com.au/results/wa/wace.aspx'

# <!-- read table -->
tables = pd.read_html(url)
tables

[     Unnamed: 0  Better Education Rank                                School  \
 0             1                      1                   Perth Modern School   
 1             2                      2  St Hilda's Anglican School for Girls   
 2             3                      3          Christ Church Grammar School   
 3             4                      4                       Penrhos College   
 4             5                      5             Methodist Ladies' College   
 ..          ...                    ...                                   ...   
 134         135                    135           Balcatta Senior High School   
 135         136                    136         Safety Bay Senior High School   
 136         137                    137          Darling Range Sports College   
 137         138                    138                       Gilmore College   
 138         139                    139                 Cecil Andrews College   
 
      Median ATAR  No. eli

In [19]:
# check table type
type(tables)

list

In [44]:
# insert table into a df and drop the first column
df = tables[0].drop(['Unnamed: 0'], axis=1)
df.head()

,Better Education Rank,School,Median ATAR,No. eligible Yr 12 students,No. Students with an ATAR,% students with an ATAR,Trend / Compare
0,1,Perth Modern School,97.55,242,242,100.00,Trend / Compare
1,2,St Hilda's Anglican School for Girls,92.70,147,134,91.16,Trend / Compare
2,3,Christ Church Grammar School,92.50,190,175,92.11,Trend / Compare
3,4,Penrhos College,90.65,135,116,85.93,Trend / Compare
4,5,Methodist Ladies' College,90.55,126,112,88.89,Trend / Compare


In [45]:
# only keep data that is useful for end user
df = df[['Better Education Rank','School','Median ATAR']]
df.head()

,Better Education Rank,School,Median ATAR
0,1,Perth Modern School,97.55
1,2,St Hilda's Anglican School for Girls,92.70
2,3,Christ Church Grammar School,92.50
3,4,Penrhos College,90.65
4,5,Methodist Ladies' College,90.55


In [65]:
cols = list(df.columns)
# rename columns to something more descriptive 
cols[0] = "2020_ATAR_Secondary_School_Ranking"
# rename School to School Name to tie in with Data Source 2
cols[1] = "School_Name"
cols[2] = "Median_ATAR"
df.columns = cols

# make the school name uppercase to enable easier merge for end user
df['School_Name'] = df['School_Name'].str.upper()

df.head()

,2020_ATAR_Secondary_School_Ranking,School_Name,Median_ATAR
0,1,PERTH MODERN SCHOOL,97.55
1,2,ST HILDA'S ANGLICAN SCHOOL FOR GIRLS,92.70
2,3,CHRIST CHURCH GRAMMAR SCHOOL,92.50
3,4,PENRHOS COLLEGE,90.65
4,5,METHODIST LADIES' COLLEGE,90.55


In [67]:
# merge in school_id
secondary_schools_df = pd.merge(df, new_WASchools_df, on="School_Name", how = "left").drop(["Classification_Group","Y12"],axis=1)
secondary_schools_df.head()

,2020_ATAR_Secondary_School_Ranking,School_Name,Median_ATAR,School_ID
0,1,PERTH MODERN SCHOOL,97.55,224.0
1,2,ST HILDA'S ANGLICAN SCHOOL FOR GIRLS,92.70,266.0
2,3,CHRIST CHURCH GRAMMAR SCHOOL,92.50,52.0
3,4,PENRHOS COLLEGE,90.65,221.0
4,5,METHODIST LADIES' COLLEGE,90.55,192.0


In [48]:
# check if there's any bad data that didn't merge
secondary_schools_df[secondary_schools_df['School_ID'].isnull()]

,2020_ATAR_Secondary_School_Ranking,School_Name,Median_ATAR,School_ID
10,11,PRESBYTERIAN LADIES' COLLEGE,89.20,NaN
48,49,"CHRISTIAN BROTHERS COLLEGE,FREMANTLE,WA,6160",82.20,NaN
61,62,"CAREY BAPTIST COLLEGE,HARRISDALE,WA,6112",80.30,NaN
101,102,AUSTRALIAN ISLAMIC COLLEGE (KEWDALE),72.55,NaN


In [49]:
# replace bad data
secondary_schools_df['School_Name'] = df['School_Name'].replace(
    {"PRESBYTERIAN LADIES' COLLEGE":"PRESBYTERIAN LADIES COLLEGE","CHRISTIAN BROTHERS COLLEGE,FREMANTLE,WA,6160":"CHRISTIAN BROTHERS' COLLEGE","CAREY BAPTIST COLLEGE,HARRISDALE,WA,6112":"CAREY BAPTIST COLLEGE","AUSTRALIAN ISLAMIC COLLEGE (KEWDALE)":"AUSTRALIAN ISLAMIC COLLEGE - KEWDALE"})

secondary_schools_df[secondary_schools_df['School_ID'].isnull()]


,2020_ATAR_Secondary_School_Ranking,School_Name,Median_ATAR,School_ID
10,11,PRESBYTERIAN LADIES COLLEGE,89.20,NaN
48,49,CHRISTIAN BROTHERS' COLLEGE,82.20,NaN
61,62,CAREY BAPTIST COLLEGE,80.30,NaN
101,102,AUSTRALIAN ISLAMIC COLLEGE - KEWDALE,72.55,NaN


In [50]:
secondary_schools_df.head()

,2020_ATAR_Secondary_School_Ranking,School_Name,Median_ATAR,School_ID
0,1,PERTH MODERN SCHOOL,97.55,224.0
1,2,ST HILDA'S ANGLICAN SCHOOL FOR GIRLS,92.70,266.0
2,3,CHRIST CHURCH GRAMMAR SCHOOL,92.50,52.0
3,4,PENRHOS COLLEGE,90.65,221.0
4,5,METHODIST LADIES' COLLEGE,90.55,192.0


In [51]:
# drop School_ID for later remerge for clean data
secondary_schools_df=secondary_schools_df.drop(["School_ID"],axis=1)
secondary_schools_df.head()

,2020_ATAR_Secondary_School_Ranking,School_Name,Median_ATAR
0,1,PERTH MODERN SCHOOL,97.55
1,2,ST HILDA'S ANGLICAN SCHOOL FOR GIRLS,92.70
2,3,CHRIST CHURCH GRAMMAR SCHOOL,92.50
3,4,PENRHOS COLLEGE,90.65
4,5,METHODIST LADIES' COLLEGE,90.55


In [57]:
# merge in school_id
secondary_schools_df2 = pd.merge(new_WASchools_df,secondary_schools_df, on="School_Name", how = "right").drop(["Classification_Group","School_Name","Y12"],axis=1)
secondary_schools_df2.head()

,School_ID,2020_ATAR_Secondary_School_Ranking,Median_ATAR
0,224,1,97.55
1,266,2,92.70
2,52,3,92.50
3,221,4,90.65
4,192,5,90.55


In [58]:
# check if there's any bad data that didn't merge
secondary_schools_df2[secondary_schools_df2['School_ID'].isnull()]

,School_ID,2020_ATAR_Secondary_School_Ranking,Median_ATAR


# LOAD

In [69]:
rds_connection_string = f"{username}:{password}@localhost:5432/schools_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [70]:
# check for tables - NOTE this is created in pgadmin before hand
engine.table_names()

<ipython-input-70-2caba68b8947>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


[]

In [ ]:
# note - delete all tables in pg admin schools_db before running the next step

In [80]:
# NOTE - DON'T RUN MORE THAN ONCE BECUASE IT APPENDS

# Dataset 1
new_WASchools_df2.to_sql(name='wa_schools_y12', con=engine, if_exists='append', index=False)

# Dataset 2
classification_df.to_sql(name='wa_schools_classification', con=engine, if_exists='append', index=False)

# Dataset 3
applecross_df2.to_sql(name='applecross_secondary_schools', con=engine, if_exists='append', index=False)

# Dataset 4
secondary_schools_df2.to_sql(name='secondary_schools_ranking', con=engine, if_exists='append', index=False)

In [81]:
# confirm Dataset 1 has been added
pd.read_sql_query('select * from wa_schools_y12', con=engine).head()

,School_ID,School_Name,Y12,Classification_ID
0,0,AL-AMEEN COLLEGE,45.0,0
1,1,ALBANY SECONDARY EDUCATION SUPPORT CENTRE,17.0,1
2,2,ALBANY SENIOR HIGH SCHOOL,130.0,2
3,3,ALKIMOS BAPTIST COLLEGE,12.0,0
4,4,ALL SAINTS' COLLEGE,144.0,0


In [82]:
# confirm Dataset 2 has been added
pd.read_sql_query('select * from wa_schools_classification', con=engine).head()

,Classification_ID,Classification_Group
0,0,NON-GOVERNMENT
1,1,EDUCATION SUPPORT
2,2,SECONDARY SCHOOLS
3,3,K-12 SCHOOLS
4,4,DISTRICT HIGH SCHOOLS


In [83]:
# confirm Dataset 3 has been added
pd.read_sql_query('select * from applecross_secondary_schools', con=engine).head()

,School_ID
0,6
1,7
2,244
3,63
4,188


In [84]:
# confirm Dataset 4 has been added
pd.read_sql_query('select * from secondary_schools_ranking', con=engine).head()

,School_ID,2020_ATAR_Secondary_School_Ranking,Median_ATAR
0,224,1,97.55
1,266,2,92.70
2,52,3,92.50
3,221,4,90.65
4,192,5,90.55
